# Introduction to Policy Gradients

This post will introduce the core concepts underlying various **policy gradient algorithms**. These algorithms are an approach to reinforcement learning utilizing **stochastic gradient ascent**.

## The Goal

Those of you familiar with neural networks will probably have heard of stochastic gradient descent. The goal of stochastic gradient descent is to calculate the gradient of a loss-function to then adjust the parameters of the network to minimize the loss-function by steping in the opposite direction of the gradient.

We can utlize the very similar stochastic gradient ascent to appraoch reinforcment learning problems. To do this, we will need a reward function $J(\theta)$ which tells us how well a given policy $\pi_\theta$ performs using the parameters $\theta$.  In reinforcment learning this is actually quite simple since reinforcment inherintly utilizes rewards $r$. Thus the reward function is chosen to be the exoected reward for a trajectory $\tau$ generated by the current policy $\pi_\theta$. Let $G(\tau)$ be the infinite horizon discounted-return starting at timestep $t = 0$ for the trajectory $\tau$ (the derivations of finite or undiscounted return are almost identical). 

\begin{align}
    G(\tau) &= \sum_{t = 1}^\infty \gamma^t r_{t+1} \\
    J(\theta) &= \mathop{\mathbb{E}}_{\tau \sim \pi_\theta} [ G(\tau) ] \\
\end{align}

If it is possible to now calculate the gradient of the reward function $J(\theta)$ we can take a stochastic gradient ascent step based on a hyperparameter $\alpha$ to slowly over many steps maximise the reward function:

\begin{align}
    \theta = \theta + \alpha \nabla_\theta J(\theta)
\end{align}

The only remaining probelem is to actually determine how the gradient $\nabla_\theta J(\theta)$ looks like.

## How to calculate the Gradient



## The goal

why use the gradient (intuition)

equation to maximise.

given a policy, form of one step


to add: curse of dimensionality, previous approaches need to analyze entire action space

## how to calculate the gradient

derive simple form of policy gradient (see open AI spinning up)

## other forms of the policy gradient

list the other typical forms of policy gradients 

policy gradient theorem

## simple policy gradient algorithm

reinforce or something similarly basic

## more advanced concepts?

baselines

TRPO

PPO

A2C